##### **Step 3: In groups, create a Jupyter notebook that shows the mortgage problem solved in Python. Make sure to:**
* Illustrate how your data structure is used to solve the problem from GWP 1.
* Ensure that your results match the results from the spreadsheet.


In [162]:
import pandas as pd
import numpy as np

pd.set_option('display.precision',2)

In [218]:
def mortgage(yearsduration, annualrate, start_date=np.nan, principal=1000000):

    ratemonthly = annualrate/12
    period = yearsduration*12+1

    principalremaining = \
        [principal * ((1+ratemonthly)**(yearsduration*12)-(1+ratemonthly)**m)/((1+ratemonthly)**(yearsduration*12)-1) \
        for m in range(period)]
    cashflow = [principal * ratemonthly/(1-1/((1+ratemonthly)**(yearsduration*12))) if m!=0 else 0 for m in range(period)]
    openningbalance = [principal]+principalremaining[:-1]
    principalpaid = np.array(openningbalance)-np.array(principalremaining)
    interestpaid = cashflow-principalpaid

    df = pd.DataFrame(columns=["Month","Opening Principal Balance","Rates","Cashflow",
                               "Interest Paid","Principal Paid","Closing Princial Balance"])
    df["Month"] = [m for m in range(period)]
    df["Rates"] = ratemonthly
    df["Cashflow"] = cashflow
    df["Closing Princial Balance"] = principalremaining
    df["Opening Principal Balance"] = openningbalance
    df["Interest Paid"] = interestpaid
    df["Principal Paid"] = principalpaid

    df.set_index('Month',inplace=True)

    return df

#### **Mortgage 1:**
* Time: 30 years
* Rate: 4%
* Principal Amount: $1,000,000
* Total Interest Paid: $718,695.06

In [220]:
Mortgage1 = mortgage(30,0.04)
Mortgage1

,Opening Principal Balance,Rates,Cashflow,Interest Paid,Principal Paid,Closing Princial Balance
Month,,,,,,
0,1000000.00,3.33e-03,0.00,0.00,0.00,1000000.00
1,1000000.00,3.33e-03,4774.15,3333.33,1440.82,998559.18
2,998559.18,3.33e-03,4774.15,3328.53,1445.62,997113.56
3,997113.56,3.33e-03,4774.15,3323.71,1450.44,995663.12
4,995663.12,3.33e-03,4774.15,3318.88,1455.28,994207.84
...,...,...,...,...,...,...
356,23633.90,3.33e-03,4774.15,78.78,4695.37,18938.53
357,18938.53,3.33e-03,4774.15,63.13,4711.02,14227.50
358,14227.50,3.33e-03,4774.15,47.43,4726.73,9500.78


#### **Mortgage 2:**
* Time: 20 years
* Rate: 2.5%
* Principal Amount: $1,000,000
* Total Interest Paid: $271,766.94

#### **Mortgage 3:**
* Time: 20 years
* Rate: 7-1 Adjustable
* Principal Amount: $1,000,000
* Total Interest Paid: $1,488,119.85